In [2]:
import gct

In [7]:
prefix = "K_DoesAlgMirrorEdgesTest"

def setup_data():
    gct.remove_data(prefix + '_*', dry_run=False)
    graph_unweighted_undirect = gct.random_dataset.generate_undirected_unweighted_random_graph_LFR(name=prefix + "_1", \
                                       N=128, k=16, maxk=32, mu=0.2, minc=32)
    data1 = graph_unweighted_undirect
    data2 = data1.as_mirror_edges(newname=prefix + "_2", overide=True)
    return data1,data2


In [8]:
data_w,data_uw=setup_data()

2020-04-22 22:21:38,746 - RandomGenerator:LFR - INFO - Runing '/opt/gct/submodules/CommunityDetectionCodes/lfr_undir_net -minc 32 -mu 0.2 -maxk 32 -k 16 -N 128'  with seed 54821
INFO:RandomGenerator:LFR:Runing '/opt/gct/submodules/CommunityDetectionCodes/lfr_undir_net -minc 32 -mu 0.2 -maxk 32 -k 16 -N 128'  with seed 54821
2020-04-22 22:21:38,747 - RandomGenerator:LFR - INFO - working dir: /tmp/tmpsf98gxe3
INFO:RandomGenerator:LFR:working dir: /tmp/tmpsf98gxe3
2020-04-22 22:21:38,778 - RandomGenerator:LFR - INFO - min node is 1. will make it 0
INFO:RandomGenerator:LFR:min node is 1. will make it 0
/opt/conda/envs/python3/lib/python3.6/site-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)
2020-04-22 22:21:38,791 - Dataset:K_DoesAlgMirrorEd

In [9]:
data_w.name,data_uw.name

('K_DoesAlgMirrorEdgesTest_1', 'K_DoesAlgMirrorEdgesTest_2')

In [10]:
import os
from gct import config,utils
def has_run_1(runame, dsname):
    fpath = os.path.join(config.get_result_file_path(dsname=dsname, runname=runame), 'result.txt')
    return utils.file_exists(fpath)

def has_run(runame, dsname):
    fpath = os.path.join("/{}/tmp/{}_{}.done".format('data', runame,dsname))
    return utils.file_exists(fpath)

def timeout(func, args=(), kwargs={}, timeout_duration=1, default=None):
    import signal

    class TimeoutError(Exception):
        pass

    def handler(signum, frame):
        raise TimeoutError()

    # set the timeout handler
    signal.signal(signal.SIGALRM, handler) 
    signal.alarm(timeout_duration)
    try:
        result = func(*args, **kwargs)
    except TimeoutError as exc:
        result = default
    finally:
        signal.alarm(0)

    return result


In [11]:

def run_alg(alg,dsname, check_skip=True):
    try:
        runname = alg
        if not check_skip or (check_skip and not has_run(runname, dsname)):
            print ("runing ", alg, dsname)
            if alg =='cdc_SVINET':
                gct.run_alg(runname=runname, data=gct.load_local_graph(dsname), algname=alg, seed=123, max_iterations=1000)        
            else:
                gct.run_alg(runname=runname, data=gct.load_local_graph(dsname), algname=alg, seed=123)
            print ("finished", alg,dsname)
        else:
            print("skip", runname,dsname)
    except:
        print("except", runname,dsname)
    return True
    

def run_alg_timeout(alg,dsname,timeout_duration=600):
    if not timeout(run_alg, (alg,dsname), timeout_duration=timeout_duration):
        print("tiemout", alg,dsname)
    fpath = os.path.join("/{}/tmp/{}_{}.done".format('data', alg,dsname))
    !touch {fpath}

        


In [12]:
run_alg("oslom_lpm", data_w.name)

2020-04-22 22:21:51,849 - Dataset:K_DoesAlgMirrorEdgesTest_1 - INFO - writing edges to /data/data/K_DoesAlgMirrorEdgesTest_1/edges.txt
INFO:Dataset:K_DoesAlgMirrorEdgesTest_1:writing edges to /data/data/K_DoesAlgMirrorEdgesTest_1/edges.txt
2020-04-22 22:21:51,850 - Dataset:K_DoesAlgMirrorEdgesTest_1 - INFO - reading /data/data/K_DoesAlgMirrorEdgesTest_1/edges.parq
INFO:Dataset:K_DoesAlgMirrorEdgesTest_1:reading /data/data/K_DoesAlgMirrorEdgesTest_1/edges.parq


runing  oslom_lpm K_DoesAlgMirrorEdgesTest_1


2020-04-22 22:21:52,059 - Dataset:K_DoesAlgMirrorEdgesTest_1 - INFO - finish writing /data/data/K_DoesAlgMirrorEdgesTest_1/edges.txt
INFO:Dataset:K_DoesAlgMirrorEdgesTest_1:finish writing /data/data/K_DoesAlgMirrorEdgesTest_1/edges.txt
2020-04-22 22:21:52,061 - oslom_lpm - INFO - Running /opt/gct/submodules/OSLOM/lpm -f /tmp/tmpmwxyj67a/edges.txt -r 1 -seed 123
INFO:oslom_lpm:Running /opt/gct/submodules/OSLOM/lpm -f /tmp/tmpmwxyj67a/edges.txt -r 1 -seed 123
2020-04-22 22:21:52,070 - oslom_lpm - INFO - Made 7 clusters in 0.008058 seconds
INFO:oslom_lpm:Made 7 clusters in 0.008058 seconds


finished oslom_lpm K_DoesAlgMirrorEdgesTest_1


True

In [14]:
run_alg_timeout("oslom_lpm", data_w.name)

skip oslom_lpm K_DoesAlgMirrorEdgesTest_1


In [15]:
datasets=[data_w.name,data_uw.name]
for alg in gct.list_algorithms():
    for name in datasets:
        run_alg_timeout(alg, name)

2020-04-22 22:22:02,906 - oslom_Infohiermap - INFO - Running /opt/gct/submodules/OSLOM/infohiermap_undir 123 /tmp/tmpc691wpqj/pajek.txt 1
INFO:oslom_Infohiermap:Running /opt/gct/submodules/OSLOM/infohiermap_undir 123 /tmp/tmpc691wpqj/pajek.txt 1
2020-04-22 22:22:02,930 - oslom_Infohiermap - INFO - Made 7 clusters in 0.013972 seconds
INFO:oslom_Infohiermap:Made 7 clusters in 0.013972 seconds


runing  oslom_Infohiermap K_DoesAlgMirrorEdgesTest_1
finished oslom_Infohiermap K_DoesAlgMirrorEdgesTest_1


2020-04-22 22:22:03,041 - Dataset:K_DoesAlgMirrorEdgesTest_2 - INFO - writing edges to /data/data/K_DoesAlgMirrorEdgesTest_2/edges.txt
INFO:Dataset:K_DoesAlgMirrorEdgesTest_2:writing edges to /data/data/K_DoesAlgMirrorEdgesTest_2/edges.txt
2020-04-22 22:22:03,043 - Dataset:K_DoesAlgMirrorEdgesTest_2 - INFO - reading /data/data/K_DoesAlgMirrorEdgesTest_2/edges.parq
INFO:Dataset:K_DoesAlgMirrorEdgesTest_2:reading /data/data/K_DoesAlgMirrorEdgesTest_2/edges.parq


runing  oslom_Infohiermap K_DoesAlgMirrorEdgesTest_2


2020-04-22 22:22:03,287 - Dataset:K_DoesAlgMirrorEdgesTest_2 - INFO - finish writing /data/data/K_DoesAlgMirrorEdgesTest_2/edges.txt
INFO:Dataset:K_DoesAlgMirrorEdgesTest_2:finish writing /data/data/K_DoesAlgMirrorEdgesTest_2/edges.txt
2020-04-22 22:22:03,304 - oslom_Infohiermap - INFO - Running /opt/gct/submodules/OSLOM/infohiermap_undir 123 /tmp/tmp3f_ep098/pajek.txt 1
INFO:oslom_Infohiermap:Running /opt/gct/submodules/OSLOM/infohiermap_undir 123 /tmp/tmp3f_ep098/pajek.txt 1
2020-04-22 22:22:03,327 - oslom_Infohiermap - INFO - Made 7 clusters in 0.013959 seconds
INFO:oslom_Infohiermap:Made 7 clusters in 0.013959 seconds


...

runing  alg_lso_cluster K_DoesAlgMirrorEdgesTest_2
finished alg_lso_cluster K_DoesAlgMirrorEdgesTest_2


2020-04-22 22:33:07,510 - alg_streamcom - INFO - Running /opt/gct/submodules/graph-streaming/streamcom --seed 123 -f /data/data/K_DoesAlgMirrorEdgesTest_1/edges.txt -o output /data/data/K_DoesAlgMirrorEdgesTest_1/edges.txt
INFO:alg_streamcom:Running /opt/gct/submodules/graph-streaming/streamcom --seed 123 -f /data/data/K_DoesAlgMirrorEdgesTest_1/edges.txt -o output /data/data/K_DoesAlgMirrorEdgesTest_1/edges.txt
2020-04-22 22:33:07,517 - alg_streamcom - INFO - Made 73 clusters in 0.005173 seconds
INFO:alg_streamcom:Made 73 clusters in 0.005173 seconds


runing  alg_streamcom K_DoesAlgMirrorEdgesTest_1
finished alg_streamcom K_DoesAlgMirrorEdgesTest_1


2020-04-22 22:33:07,628 - alg_streamcom - INFO - Running /opt/gct/submodules/graph-streaming/streamcom --seed 123 -f /data/data/K_DoesAlgMirrorEdgesTest_2/edges.txt -o output /data/data/K_DoesAlgMirrorEdgesTest_2/edges.txt
INFO:alg_streamcom:Running /opt/gct/submodules/graph-streaming/streamcom --seed 123 -f /data/data/K_DoesAlgMirrorEdgesTest_2/edges.txt -o output /data/data/K_DoesAlgMirrorEdgesTest_2/edges.txt
2020-04-22 22:33:07,636 - alg_streamcom - INFO - Made 70 clusters in 0.005772 seconds
INFO:alg_streamcom:Made 70 clusters in 0.005772 seconds


runing  alg_streamcom K_DoesAlgMirrorEdgesTest_2
finished alg_streamcom K_DoesAlgMirrorEdgesTest_2


In [27]:
    results = {}
    d = gct.list_all_clustering_results(print_format=False)
    for u in d:
        if prefix + "_" in u:
            for run in d[u]:
                a = gct.load_clustering_result(u, run)
                print (u, run)
                results[u + run] = a



K_DoesAlgMirrorEdgesTest_1 oslom_lpm
K_DoesAlgMirrorEdgesTest_1 oslom_Infohiermap
K_DoesAlgMirrorEdgesTest_1 oslom_Infomap
K_DoesAlgMirrorEdgesTest_1 oslom_OSLOM
K_DoesAlgMirrorEdgesTest_1 oslom_copra
K_DoesAlgMirrorEdgesTest_1 oslom_louvain_method
K_DoesAlgMirrorEdgesTest_1 oslom_modopt
K_DoesAlgMirrorEdgesTest_1 pycabem_GANXiSw
K_DoesAlgMirrorEdgesTest_1 pycabem_HiReCS
K_DoesAlgMirrorEdgesTest_1 pycabem_LabelRank
K_DoesAlgMirrorEdgesTest_1 cdc_CONGA
K_DoesAlgMirrorEdgesTest_1 cdc_CliquePercolation
K_DoesAlgMirrorEdgesTest_1 cdc_Connected_Iterative_Scan
K_DoesAlgMirrorEdgesTest_1 cdc_EAGLE
K_DoesAlgMirrorEdgesTest_1 cdc_FastCpm
K_DoesAlgMirrorEdgesTest_1 cdc_GCE
K_DoesAlgMirrorEdgesTest_1 cdc_LinkCommunities
K_DoesAlgMirrorEdgesTest_1 cdc_MOSES
K_DoesAlgMirrorEdgesTest_1 cdc_MSCD_AFG
K_DoesAlgMirrorEdgesTest_1 cdc_MSCD_HSLSW
K_DoesAlgMirrorEdgesTest_1 cdc_MSCD_LFK
K_DoesAlgMirrorEdgesTest_1 cdc_MSCD_LFK2
K_DoesAlgMirrorEdgesTest_1 cdc_MSCD_RB
K_DoesAlgMirrorEdgesTest_1 cdc_MSCD_RN
K_D

In [28]:
import pandas as pd
lst = [];
for alg in  gct.list_algorithms():
    for dsname in datasets:
        if dsname.startswith(prefix + '_1'):
            a = [];columns = []
            dsname2 = dsname.replace(prefix + "_1", prefix + "_2")
            a += [alg, dsname, dsname2]
            columns += ['alg', 'data1', 'data2']
            if dsname + alg in results and dsname2 + alg in results:
                cluster1 = gct.to_cluster(results[dsname + alg])
                cluster2 = gct.to_cluster(results[dsname2 + alg])
                compa = gct.ClusterComparator(cluster1, cluster2)
                a += [compa.sklean_nmi(), cluster1.is_overlap, cluster2.is_overlap,
                      cluster1.num_cluster, cluster2.num_cluster,
                     compa.OvpNMI()['NMImax']]
                columns += ['nmi_12', 'ovp1', 'ovp2', "#c1", "#c2", 'ovpnmi_12']

                gt = list(gct.load_local_graph(dsname).get_ground_truth().values())[0]            
                compa = gct.ClusterComparator(gt, cluster1)            
                a += [compa.sklean_nmi(), compa.OvpNMI()['NMImax']]
                columns += ['nmi_t1', 'ovpnmi_t1']

                compa = gct.ClusterComparator(gt, cluster2)            
                a += [compa.sklean_nmi(), compa.OvpNMI()['NMImax']]
                columns += ['nmi_t2', 'ovpnmi_t2']

                lst.append(a)



2020-04-23 00:49:45,572 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-23 00:49:45,582 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpeg6ocsyw/cluster1.cnl /tmp/tmpeg6ocsyw/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpeg6ocsyw/cluster1.cnl /tmp/tmpeg6ocsyw/cluster2.cnl > ovpnmioutput
2020-04-23 00:49:45,591 - Clustering - INFO - reading/data/data/K_DoesAlgMirrorEdgesTest_1/gt_default.parq
INFO:Clustering:reading/data/data/K_DoesAlgMirrorEdgesTest_1/gt_default.parq
2020-04-23 00:49:45,598 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INF

INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpyzlg5vfr/cluster1.cnl /tmp/tmpyzlg5vfr/cluster2.cnl > ovpnmioutput
2020-04-23 00:49:46,331 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-23 00:49:46,339 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpz1up20ho/cluster1.cnl /tmp/tmpz1up20ho/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpz1up20ho/cluster1.cnl /tmp/tmpz1up20ho/cluster2.cnl > ovpnmioutput
2020-04-23 00:49:46,356 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of

2020-04-23 00:49:46,564 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpqqxsk6e2/cluster1.cnl /tmp/tmpqqxsk6e2/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpqqxsk6e2/cluster1.cnl /tmp/tmpqqxsk6e2/cluster2.cnl > ovpnmioutput
2020-04-23 00:49:46,573 - Clustering - INFO - reading/data/data/K_DoesAlgMirrorEdgesTest_1/gt_default.parq
INFO:Clustering:reading/data/data/K_DoesAlgMirrorEdgesTest_1/gt_default.parq
2020-04-23 00:49:46,579 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-23 00:49:46,588 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onm

INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpgjzvpeqc/cluster1.cnl /tmp/tmpgjzvpeqc/cluster2.cnl > ovpnmioutput
2020-04-23 00:49:47,177 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-23 00:49:47,185 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp5_ouclej/cluster1.cnl /tmp/tmp5_ouclej/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp5_ouclej/cluster1.cnl /tmp/tmp5_ouclej/cluster2.cnl > ovpnmioutput
2020-04-23 00:49:47,408 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of

2020-04-23 00:49:48,689 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-23 00:49:48,699 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpx_nd4zr3/cluster1.cnl /tmp/tmpx_nd4zr3/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpx_nd4zr3/cluster1.cnl /tmp/tmpx_nd4zr3/cluster2.cnl > ovpnmioutput
2020-04-23 00:49:48,708 - Clustering - INFO - reading/data/data/K_DoesAlgMirrorEdgesTest_1/gt_default.parq
INFO:Clustering:reading/data/data/K_DoesAlgMirrorEdgesTest_1/gt_default.parq
2020-04-23 00:49:48,715 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INF

...

INFO:ClusterComparator:resulting 128 nodes out of 128,128
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-23 00:49:53,289 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpd77yyv3d/cluster1.cnl /tmp/tmpd77yyv3d/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpd77yyv3d/cluster1.cnl /tmp/tmpd77yyv3d/cluster2.cnl > ovpnmioutput
2020-04-23 00:49:53,298 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use a

INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpefvtc7b4/cluster1.cnl /tmp/tmpefvtc7b4/cluster2.cnl > ovpnmioutput
2020-04-23 00:49:53,524 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-23 00:49:53,534 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpwgqbyl6q/cluster1.cnl /tmp/tmpwgqbyl6q/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpwgqbyl6q/cluster1.cnl /tmp/tmpwgqbyl6q/cluster2.cnl > ovpnmioutput
2020-04-23 00:49:53,543 - Clustering - INFO - reading/data/data/K_DoesAlgMirrorEdgesTest_1/gt_default.parq
INFO:Clustering:reading/data/d

INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpym_4kam9/cluster1.cnl /tmp/tmpym_4kam9/cluster2.cnl > ovpnmioutput
2020-04-23 00:49:53,752 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-23 00:49:53,762 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp3anchre0/cluster1.cnl /tmp/tmp3anchre0/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp3anchre0/cluster1.cnl /tmp/tmp3anchre0/cluster2.cnl > ovpnmioutput
2020-04-23 00:49:53,771 - Clustering - INFO - reading/data/data/K_DoesAlgMirrorEdgesTest_1/gt_default.parq
INFO:Clustering:reading/data/d

INFO:ClusterComparator:resulting 128 nodes out of 128,128
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-23 00:49:53,985 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpozoi6fhn/cluster1.cnl /tmp/tmpozoi6fhn/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpozoi6fhn/cluster1.cnl /tmp/tmpozoi6fhn/cluster2.cnl > ovpnmioutput
2020-04-23 00:49:53,993 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use a

INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpm92vd3nv/cluster1.cnl /tmp/tmpm92vd3nv/cluster2.cnl > ovpnmioutput
2020-04-23 00:49:54,219 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-23 00:49:54,241 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp1wiipu5f/cluster1.cnl /tmp/tmp1wiipu5f/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp1wiipu5f/cluster1.cnl /tmp/tmp1wiipu5f/cluster2.cnl > ovpnmioutput
2020-04-23 00:49:54,250 - Clustering - INFO - reading/data/data/K_DoesAlgMirrorEdgesTest_1/gt_default.parq
INFO:Clustering:reading/data/d

In [29]:
rdf = pd.DataFrame(lst, columns=columns)
for alg in  gct.list_algorithms():
    if alg not in rdf['alg'].values:
        lst.append([alg] + [None] * (len(columns) - 1))
rdf = pd.DataFrame(lst, columns=columns)    
with pd.option_context('display.max_rows', 2000, 'display.max_columns', 200):
    display (rdf)


,alg,data1,data2,nmi_12,ovp1,ovp2,#c1,#c2,ovpnmi_12,nmi_t1,ovpnmi_t1,nmi_t2,ovpnmi_t2
0,oslom_Infohiermap,K_DoesAlgMirrorEdgesTest_1,K_DoesAlgMirrorEdgesTest_2,1.000000,False,False,7.0,7.0,1.000000,1.000000,1.000000,1.000000,1.000000
1,oslom_Infomap,K_DoesAlgMirrorEdgesTest_1,K_DoesAlgMirrorEdgesTest_2,0.939308,False,False,7.0,7.0,0.910230,0.939308,0.910230,1.000000,1.000000
2,oslom_OSLOM,K_DoesAlgMirrorEdgesTest_1,K_DoesAlgMirrorEdgesTest_2,1.000000,False,False,7.0,7.0,1.000000,1.000000,1.000000,1.000000,1.000000
3,oslom_copra,K_DoesAlgMirrorEdgesTest_1,K_DoesAlgMirrorEdgesTest_2,0.928029,True,True,7.0,7.0,0.927268,0.983885,0.984499,0.983559,0.938501
4,oslom_louvain_method,K_DoesAlgMirrorEdgesTest_1,K_DoesAlgMirrorEdgesTest_2,1.000000,False,False,7.0,7.0,1.000000,1.000000,1.000000,1.000000,1.000000
5,oslom_lpm,K_DoesAlgMirrorEdgesTest_1,K_DoesAlgMirrorEdgesTest_2,1.000000,False,False,7.0,7.0,1.000000,1.000000,1.000000,1.000000,1.000000
6,oslom_modopt,K_DoesAlgMirrorEdgesTest_1,K_DoesAlgMirrorEdgesTest_2,1.000000,False,False,7.0,7.0,1.000000,1.000000,1.000000,1.000000,1.000000
7,pycabem_GANXiSw,K_DoesAlgMirrorEdgesTest_1,K_DoesAlgMirrorEdgesTest_2,1.000000,False,False,6.0,6.0,1.000000,0.940355,0.843600,0.940355,0.843600
8,pycabem_HiReCS,K_DoesAlgMirrorEdgesTest_1,K_DoesAlgMirrorEdgesTest_2,1.000000,False,False,7.0,7.0,1.000000,0.152460,NaN,0.152460,NaN
9,pycabem_LabelRank,K_DoesAlgMirrorEdgesTest_1,K_DoesAlgMirrorEdgesTest_2,1.000000,False,False,7.0,7.0,1.000000,1.000000,1.000000,1.000000,1.000000


In [26]:
run_alg("cdc_CliquePercolation", "K_DoesAlgMirrorEdgesTest_2", check_skip=False)

runing  cdc_CliquePercolation K_DoesAlgMirrorEdgesTest_2
finished cdc_CliquePercolation K_DoesAlgMirrorEdgesTest_2


Error: only undirected and unmirrored graph is supported for CliquePercolation


True